<a href="https://colab.research.google.com/github/mandip-openai/SourceUtilityAG/blob/main/Source_Utility_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graders


In [ ]:
from maraschino_rater.q2d.raters import make_pairwise_rater, make_pointwise_rater
import maraschino_rater.utils.lm_utils as lm_utils
import json
import functools
import asyncio
import typing
from maraschino_rater.base import (
    Document,
    PairwiseRater,
    PairwiseRating,
    PointwiseRater,
    PointwiseRating,
    Query,
)
from pprint import pprint

DEFAULT_INSTRUCTION = f"""
You are a professional grader that judges the quality of search results.
You need to follow the rating instruction below to rate a given QUERY - DOCUMENT pair with respect to each dimension.
The QUERY contains the QUERY TEXT, DATE, and LOCATION of the searcher. The DOCUMENT contains TITLE, URL, PUBLICATION DATE, and CONTENT.
Write your decision for each aspects in the markdown format. It should contain a title of the area, the rating, and detailed justification of your decision.
If you have any confusion or something worth commenting, can also output them in a "notes".

The response of each aspect (dimension) is:
## <Aspect>
* rating: <rating>
* justification: <justification>
* notes: <notes>

The Rubrics is:

"""

DEFAULT_RUBRICS = "Rate the relevance of the the search result DOCUMENT to the user QUERY."

class Grader:
  def __init__(self, instruction: str = DEFAULT_INSTRUCTION):
    self.instruction = instruction

  async def __call__(self, query: Query, document: Document, rubrics: str = ""):
    date_spec = ""
    if query.create_timestamp:
        date_spec = f"{query.create_timestamp.date()}"
    geo_spec = ""
    if (
        query.user_metadata
        and query.user_metadata.user_country
        and query.user_metadata.user_region
        and query.user_metadata.ip_city
    ):
        geo_spec = f"country - {query.user_metadata.user_country}, region - {query.user_metadata.user_region}, city - {query.user_metadata.ip_city}"

    input = f"""
# Input Query and Document

QUERY:
QUERY TEXT: {query.query}
DATE: {date_spec or "N/A"}
LOCATION: {geo_spec or "N/A"}

DOCUMENT:
TITLE: {document.title or "N/A"}
URL: {document.url}
PUBLICATION DATE: {document.pub_date or "N/A"}
CONTENT: {document.content if document.content else "N/A"}
    """

    return await self.rate_text(input, rubrics)

  async def rate_text(self, text: str, rubrics: str):
    raise NotImplementedError()

# This is DEMO, do not take it seriously
def gen_example():
  q = Query(query="Liberal Democrats largest donor 2005")

  d = Document(
      url="https://www.theguardian.com/politics/2017/jun/15/controversial-lib-dem-donor-gave-200000-to-election-campaign",
      title="Controversial Lib Dem donor gave £200,000 to election campaign | Party funding | The Guardian",
      content="""
  In January, Rolls-Royce agreed to pay 【15†£671m in penalties】 to settle long-running bribery investigations by the Serious Fraud Office and overseas prosecutors, but separate criminal inquiries may follow in countries around the world where the company has admitted corruption.

  The Guardian and the BBC’s Panorama have previously revealed that Choudhrie family companies were paid millions by Rolls-Royce for their work in 【16†India】.

  In a letter, the Choudhries’ lawyers said: “Mr Choudhrie has never paid bribes to government officials or acted as an illegal ‘middleman’ in defence deals.”

  Donors to the Tories included Express Newspapers, which gave nearly £80,000 in free advertising. The newspaper’s proprietor, Richard Desmond, backed Ukip in 2015 with a £1m donation, but his paper’s support for the anti-European party has waned since Britain voted to leave the EU.

  Two of David Cameron’s most important donors also gave money in the last week of the campaign. Ian Taylor, the founder of oil trader Vitol, gave £25,000. Taylor withdrew his name from consideration for a knighthood in Cameron’s resignation honours list last year. Questions had been raised about Vitol’s activities in politically sensitive areas around the world. Taylor has been a Conservative donor for several years, and supported the remain campaign.

  Countywide Developments, a property company run by Tony Gallagher, gave £25,000 in the last week. At the end of last year, Cameron celebrated his 50th birthday at Gallagher’s 17th-century Oxfordshire mansion.

  David Rowland, a billionaire property developer, gave £100,000 to the May campaign.

  Labour received £10,000 from the comedian Eddie Izzard and £8,000 from Matt Wrack, the head of the Fire Brigades Union.

  Explore more on these topics.
      """
  )

  rubics = DEFAULT_RUBRICS
  return q, d, rubics


## Existing Grader

* Do not rely on this too much for now as the grader engine is occupied by backfilling.

In [ ]:
from maraschino_rater.q2d.raters import make_pairwise_rater, make_pointwise_rater
from maraschino_rater.utils.lm_utils import RetriableLM, lm_from_api, neutrino_from_completer


# lm_from_api(model=model, max_concurrency=max_concurrency)


class ExistingGrader(Grader):
  def __init__(self, instruction=DEFAULT_INSTRUCTION):
    super().__init__(instruction)
    self.grader = make_pointwise_rater(
        name="neutrino_pointwise_rater",
        lm=neutrino_from_completer(topic_mode_or_user="bwen-citation-neutrino-grader-0116"),
    )

  async def __call__(self, query: Query, document: Document):
    rating = await self.grader(query, document)
    return rating



async def test_existing_grader():
  q, d, _ = gen_example()
  grader = ExistingGrader()
  rating = await grader(q, d)
  print(rating)

# await test_existing_grader()

## Arbitrary Grader

In [ ]:
def _get_lm(
      model: typing.Literal["neutrino", "gpt-4o-mini", "gpt4o-sonic-training"],
  ) -> lm_utils.LanguageModel:
      if model == "neutrino":
          lm = lm_utils.neutrino_from_completer(topic_mode_or_user="bwen-citation-neutrino-grader-0116")
      else:
          lm = lm_utils.lm_from_api(model=model)

      return lm_utils.RetriableLM(
          lm=lm,
          max_retry=3,
          max_concurrency=10,
      )


class GeneralGrader(Grader):
  def __init__(self, model: str, instruction=DEFAULT_INSTRUCTION):
    super().__init__(instruction)
    self.grader = _get_lm(model=model)

  async def rate_text(self, text: str, rubrics: str):
    conv = [
        lm_utils.system_message(
            f"""You are ChatGPT, a large language model developed by OpenAI.
          """
        ),
        lm_utils.user_message("\n".join([self.instruction, rubrics, text])),
    ]

    ans = None
    try:
      ans = await self.grader(conv)
      if ans.content is None:
          raise ValueError("Cannot parse empty message content")
      return ans.content
    except Exception as e:
        return f"Error {e}, {str(ans)}"


async def test_general_grader(model="neutrino"):
  q, d, rubrics = gen_example()
  grader = GeneralGrader(model=model)
  rating = await grader(q, d, rubrics)
  print(rating)

await test_general_grader("gpt-4o-mini")


## Relevance
* rating: 2
* justification: The document discusses donations to the Liberal Democrats, but it does not specifically address the largest donor in 2005, which is the focus of the query. While it mentions a significant donation to the party, it lacks the specific information requested by the user regarding the largest donor for that particular year. The content is somewhat related but does not fulfill the user's request adequately.
* notes: The document could be more relevant if it included specific details about the largest donor to the Liberal Democrats in 2005, as that is the key aspect of the query. 

## Authority
* rating: 4
* justification: The document is from The Guardian, a reputable news organization known for its political coverage. This adds credibility to the information presented, even though it does not fully address the query.
* notes: The publication date is not provided, which could affect the perceived timeliness of the information, but The Guardian's esta

# TB grader

In [ ]:
from sonic_training.experiments.index_switcher.v2.grader import utils

class ToolberryGrader(Grader):
  async def rate_text(self, text: str, rubrics: str):
    convo = await utils.query_toolberry(prompt="\n".join([self.instruction, rubrics, text]), reward_multipler=128)
    resp = str(convo.messages[-1].content)
    return resp

async def test_tbv2_grader():
  q, d, rubrics = gen_example()
  grader = ToolberryGrader()
  rating = await grader(q, d, rubrics)
  print(rating)

await test_tbv2_grader()

[2025-01-17T07:41:40.241025Z][syncio:19][I] Starting event loop canary
[2025-01-17T07:41:40.361456Z][otel_utils:37][W] OTel metrics are not initialized, please initialize in your application by calling `otel.metrics.init()`
[2025-01-17T07:41:40.561119Z][lazy_cluster:824][I] Starting RedisClient pool for berry-shared-default: host=bus-west-2.westus.redisenterprise.cache.azure.net, port=10000, max_connections=4
[2025-01-17T07:41:41.673134Z][lazy_cluster:824][I] Starting RedisClient pool for berry-shared-liveness: host=bus-west-2.westus.redisenterprise.cache.azure.net, port=10000, max_connections=2
[2025-01-17T07:41:42.538872Z][lazy_cluster:824][I] Starting RedisClient pool for berry-shared-block_read: host=bus-west-2.westus.redisenterprise.cache.azure.net, port=10000, max_connections=6
[2025-01-17T07:41:42.578578Z][_simple_requestor:31][I] Request berry-shared:client:bwen-quail-quail-spark-master-0-435Q9p:0 submitted to berry-shared:snap:oaistrawberrychkpts/checkpoints/strawberrybr-tbv2-

# TB with Browse

In [ ]:
from sonic_training.experiments.tools.lean_browser_sampling import load_completer, make_convo

class ToolberryWithBrowseGrader(Grader):
  async def rate_text(self, text: str, rubrics: str):
    convo = make_convo(
      "\n".join(
          [self.instruction, rubrics, text, "You can use browser.tool_call to help you grading. "]
      )
    )
    tc = await load_completer()
    result = None
    async for result in tc.async_completion_stream(convo, include_system_messages=False):
        pass
    assert result is not None
    for message in result.input_conversation.messages:
        print(f"input: {message=}")
    for message in result.output_messages:
        print(f"output: {message=}")
    return result.output_messages[-1]

async def test_tbv2_with_browse_grader():
  q, d, rubrics = gen_example()
  grader = ToolberryWithBrowseGrader()
  rating = await grader(q, d, rubrics)
  print(rating)

await test_tbv2_with_browse_grader()

2025-01-17 22:49:30 [debug    ] Successfully subscribed to berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW. bus=True
2025-01-17 22:49:32 [warning  ] %s: %s outstanding. %s         bus=True positional_args=('berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW', "seq #'s 6 (1737154171247-0)", '0 pending/1 outstanding/6 completed/0 errors')
2025-01-17 22:49:32 [warning  ] %s: no more outstanding/pending requests; closing requestor. %s bus=True positional_args=('berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW', '0 pending/0 outstanding/7 completed/0 errors')
2025-01-17 22:49:32 [debug    ] TopicManager(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW): 'CancelledError - Requestor(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW).close()' thrown; exiting. bus=True


2025-01-17 22:49:32 [debug    ] _Requestor.handle_responses(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW): 'CancelledError - Requestor(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW).close()' thrown; exiting. bus=True


2025-01-17 22:49:32 [debug    ] HeartbeatManager(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW:alive): 'CancelledError - HeartbeatManager(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW:alive).stop()' thrown; exiting. bus=True


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ content_type='system_error' name='ToolUsageError' text='No pages to access!'                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
Content Type: system_error
Channel: analysis
Weight: 1.0
End Turn: None



2025-01-17 22:49:32 [info     ] Model function call complete   component=function_calling_tool function_calling_tool={'tool_name': 'browser', 'call_id': 'fncall_86c3ec6aea9b42259dad7fb928a5ad12', 'input_message': {'id': UUID('e781f5aa-04d6-4a99-835e-e2075ae8994d'), 'author': {'role': <Role.ASSISTANT: 'assistant'>, 'name': None}, 'recipient': 'browser.open', 'channel': 'analysis', 'content': {'content_type': 'code', '__str__': '{"id": 0, "cursor": 0}'}}, 'response_message_ids': ['ead61718-a2dc-4f5b-96a0-60f8fd0b4e39'], 'finished_successfully': True, 'function_calling_tool_version': '1.0.2', 'function_name': 'open', 'kwargs': {'id': 0, 'cursor': 0}, '__fn_calling_tool_fn_type__': 'function_the_model_can_call'}
2025-01-17 22:49:32 [debug    ] Successfully subscribed to berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW. bus=True
2025-01-17 22:49:33 [warning  ] %s: no more outstanding/pending requests; closing requestor. %s bus=True positional_args=('berry-shared:client:bwen-quail

...

2025-01-17 22:50:16 [debug    ] _Requestor.handle_responses(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW): 'CancelledError - Requestor(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW).close()' thrown; exiting. bus=True


2025-01-17 22:50:16 [debug    ] HeartbeatManager(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW:alive): 'CancelledError - HeartbeatManager(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW:alive).stop()' thrown; exiting. bus=True


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ content_type='tether_browsing_display' result="L0: # 【0†General Election 2010: Lib Dems received £100,000      │
│ donation from 'non \nL1: ...†www.telegraph.co.uk】\nL2: Another controversial Lib Dem donor is Pheroze          │
│ Sorabjee, who donated £10,000 to \nL3: the party last month. He works for Jeffcote Donnison LLP, a tax          │
│ consultancy firm\nL4:  which advises on offshore tax ..." summary='Search results for query `"Controversial Lib │
│ Dem donor"`\n**viewing lines [0 - 4] of 4**\n\n' assets=None tether_id=1                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
Content Type: tether_browsing_display
Channel: analysis
Weight: 1.0
End Turn: None



2025-01-17 22:50:16 [info     ] Model function call complete   component=function_calling_tool function_calling_tool={'tool_name': 'browser', 'call_id': 'fncall_203ab61c12964344afecaafd1b89162d', 'input_message': {'id': UUID('b4eeffaf-4018-4b7e-a788-acc427c85f32'), 'author': {'role': <Role.ASSISTANT: 'assistant'>, 'name': None}, 'recipient': 'browser.search', 'channel': 'analysis', 'content': {'content_type': 'code', '__str__': '{"query": "\\"Controversial Lib Dem donor\\""}'}}, 'response_message_ids': ['c41eacd2-d5ca-4ea9-b0fe-e3971747e683'], 'finished_successfully': True, 'function_calling_tool_version': '1.0.2', 'function_name': 'search', 'kwargs': {'query': '"Controversial Lib Dem donor"'}, '__fn_calling_tool_fn_type__': 'function_the_model_can_call'}
2025-01-17 22:50:16 [debug    ] Successfully subscribed to berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW. bus=True
2025-01-17 22:50:17 [warning  ] %s: no more outstanding/pending requests; closing requestor. %s bus=True 

assistant -> all
╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ## Relevance                                                                                                    │
│ * rating: Irrelevant                                                                                            │
│ * justification: The query asks about the Liberal Democrats' largest donor in 2005, and the document is a       │
│ Guardian article from 2017 about various donors, not specifically about the largest donor in 2005. It does not  │
│ answer the query.                                                                                               │
│ * notes: The content does not mention 2005 or the specific donor sought.                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
Content Type: text
Weight: 1.0
message_content_token_co

# Neutrino V4

* Model ckpts: https://openai.enterprise.slack.com/archives/D083FD1C34Y (copied to centralus)
* Model config (from baseline model): https://sourcegraph.internal.api.openai.org/github.com/openai/openai/-/blob/snapshot-store/snapshot_store/torchflow/_scallion_lpe_snapshots.py?L19:28-19:138
* Extra config: https://sourcegraph.internal.api.openai.org/github.com/openai/openai/-/blob/personal/hthu/lpe/export_lpe.sh?L15
* API Call https://sourcegraph.internal.api.openai.org/github.com/openai/openai@3bf148a/-/blob/personal/maja/maja/try_nv4.py

```
oaipkg run harmony_scripts.engines.start_engine \
  --model_config=falcon.multimodal.runs.scallion-d36-s64-lpe \
  --extra_config_string="twppo.scallion.lpe.common twppo.sampling_only" \
  --mode=optimal \
  --n_replicas=1 \
  --snapshot_path=az://oaidsm2/oaistrawberry2/twapi/mini/e/bmckinzie-nv4-25T-mident-ipb512-spi128-tbv2-run1/policy/step_000320/ \
  --cluster=viper --team=posttraining \
  --priority=team-critical \
  --renderer_name=harmony_v4.0.15_berry_v3_1mil_orion_lpe \
  --use_bus_v2=True --name=neutrino-v4-test \
  --num_copies=1 \
  --bus_topic_mode_or_user=bwen-citation-neutrinov4-test \
  --skip_user_config=False \
  --force
  ```


In [ ]:
from sonic_training.experiments.index_switcher.v2.grader import utils
from chat import chat


from chat.render.v4.experimental.strawberry.formatter import BerryChannel
from message_completer.berry_selective_message_completer import BerrySelectiveMessageCompleter
from bus_token_completer import BusTokenCompleter
from turn_completer import BerryMultiMessageTurnCompleter, TurnCompleter


# Dont work
# def get_multiturn_completer():
#   berry_message_completer_config = BerryMultiMessageTurnCompleter.Config(
#       token_completer_config=BusTokenCompleter.Config(
#           # assumes you spinned up this engine
#           topic_or_snapshot="az://oaidsm2/oaistrawberry2/twapi/mini/e/bmckinzie-nv4-25T-mident-ipb512-spi128-tbv2-run1/policy/step_000320/",
#           topic_mode_or_user="bwen-citation-neutrinov4-test",
#       ),
#       completion_params={"temperature": 0.95, "top_p": 0.995},
#       renderer="harmony_v4.0.15_berry_v3_1mil_orion_lpe",
#   )
#   berry_message_completer = berry_message_completer_config.build()
#   return berry_message_completer

def get_selective_completer():
  berry_message_completer_config = BerrySelectiveMessageCompleter.Config(
    token_completer_config=BusTokenCompleter.Config(
        # assumes you spinned up this engine
        topic_or_snapshot="az://oaidsm2/oaistrawberry2/twapi/mini/e/bmckinzie-nv4-25T-mident-ipb512-spi128-tbv2-run1/policy/step_000320/",
        topic_mode_or_user="bwen-citation-neutrinov4-test",
    ),
    completion_params={"temperature": 1, "top_p": 0.995, "max_tokens": 64000},
    renderer="harmony_v4.0.15_berry_v3_1mil_orion_lpe",
  )
  berry_message_completer = berry_message_completer_config.build()
  return berry_message_completer


class NeutrinoV4Grader(Grader):
  def __init__(self, instruction=DEFAULT_INSTRUCTION):
    super().__init__(instruction)
    self.completer = get_selective_completer()

  async def rate_text(self, text: str, rubrics: str):
    convo = chat.Conversation(
        messages=[
            chat.Message.system("You are ChatGPT."),
            chat.Message(role=chat.Role.USER, content=chat.MultimodalText(parts=[self.instruction, rubrics, text])),
        ]
    )
    completion = self.completer.completion(
        [convo],
        channel_selector=BerryChannel.FINAL_ANSWER,
        valid_channels=[BerryChannel.CHAIN_OF_THOUGHT, BerryChannel.FINAL_ANSWER],
        reward_multiplier=256,
        return_all_messages=True,
    )

    print(completion.choices[0])
    return completion.choices[0].output_conversation.messages[-1].content.parts[0]

    # convo = utils.init_berry_convo(prompt="\n".join([self.instruction, rubrics, text]), reward_multipler=128)
    # completion = await utils.toolberry_completion(convo, max_tokens=64000, completer=self.completer)
    # resp = str(completion.messages[-1].content)
    # return resp


async def test_nv4_grader():
  q, d, rubrics = gen_example()
  grader = NeutrinoV4Grader()
  rating = await grader(q, d, rubrics)
  print(rating)

await test_nv4_grader()

2025-01-18 06:50:12 [debug    ] Successfully subscribed to bus:client:bwen-quail-spart-grader-master-0-8jeRH9. bus=True
2025-01-18 06:50:17 [warning  ] %s: %s outstanding. %s         bus=True positional_args=('bus:client:bwen-quail-spart-grader-master-0-8jeRH9', "seq #'s 2 (1737183012562-0)", '0 pending/1 outstanding/2 completed/0 errors')
2025-01-18 06:50:17 [warning  ] %s: no more outstanding/pending requests; closing requestor. %s bus=True positional_args=('bus:client:bwen-quail-spart-grader-master-0-8jeRH9', '0 pending/0 outstanding/3 completed/0 errors')
2025-01-18 06:50:17 [debug    ] TopicManager(bus:client:bwen-quail-spart-grader-master-0-8jeRH9): 'CancelledError - Requestor(bus:client:bwen-quail-spart-grader-master-0-8jeRH9).close()' thrown; exiting. bus=True
2025-01-18 06:50:17 [debug    ] _Requestor.handle_responses(bus:client:bwen-quail-spart-grader-master-0-8jeRH9): 'CancelledError - Requestor(bus:client:bwen-quail-spart-grader-master-0-8jeRH9).close()' thrown; exiting. bu

# NV4 with browser

In [ ]:
from sonic_training.experiments.tools.lean_browser_sampling import HarmonyLeanBrowserToolFactory, make_convo, HarmonyLeanBrowserTool, init_tool
from chatgpt_agents.chatgpt_agent import ChatGPTAgent
from harmony_agent_turn_completer import HarmonyAgentTurnCompleter, agent_factories
from agent_core import ConfiguredAgent, ModelFamily
from token_completer.token_completer import TokenCompleter
from syncio import SyncClientHelper
from message_completer import TokenMessageCompleter
from bus_token_completer import BusTokenCompleter

#!! I feel nv4 tool use of browse is not tuned well. it generates bad inputs to the tool
def load_completer():
  token_completer=BusTokenCompleter.Config(
      # assumes you spinned up this engine
      topic_or_snapshot="az://oaidsm2/oaistrawberry2/twapi/mini/e/bmckinzie-nv4-25T-mident-ipb512-spi128-tbv2-run1/policy/step_000320/",
      topic_mode_or_user="bwen-citation-neutrinov4-test",
  ).build()
  tmc = TokenMessageCompleter(
      token_completer=token_completer,
      completion_params=TokenCompleter.Params(
          max_tokens=64000,
          temperature=1.0,
          top_p=0.995,
      ),
      renderer="harmony_v4.0.15_berry_v3_1mil_orion_lpe",
  )
  # tmc = get_selective_completer()

  browser_tool_factory = HarmonyLeanBrowserToolFactory()
  return HarmonyAgentTurnCompleter(
      configured_agent_factory=lambda: ConfiguredAgent(
          type=ChatGPTAgent,
          settings=agent_factories.harmony_agent_settings(
              renderer_name="harmony_v4.0.15_berry_v3_1mil_orion_lpe",
              api_base="http://should-not-be-used.openai.com/",
              agent_type=ChatGPTAgent,
              model_family=ModelFamily.UNKNOWN,
              default_tools=[
                  HarmonyLeanBrowserTool().serialize(),
              ],
              max_consecutive_model_messages=5,
              vardisc_reward_multiplier=16,
              sampling_temperature=1.0,
              # extra_settings
              sampling_max_tokens=64000,
              allow_wasted_action_budget=True,
              sampling_top_p=0.995,
          ),
      ),
      agent_context_factory=agent_factories.AgentContextFactoryFactory(
          message_completer=tmc,
          own_message_completer=True,
          tool_context_factories={browser_tool_factory.context_type: browser_tool_factory},
      ),
      sync_helper=SyncClientHelper(max_concurrency=None),
  )


class NeutrinoV4WithBrowseGrader(Grader):
  async def rate_text(self, text: str, rubrics: str):
    convo = make_convo(
      "\n".join(
          [
              self.instruction,
              rubrics,
              text,
              "You can use browser.tool_call to help you grading. "
          ]
      )
    )
    tc = load_completer()
    result = None
    async for result in tc.async_completion_stream(convo, include_system_messages=False):
        pass
    assert result is not None
    for message in result.input_conversation.messages:
        print(f"input: {message=}")
    for message in result.output_messages:
        print(f"output: {message=}")
    return result.output_messages[-1]

async def test_nv4_with_browse_grader():
  q, d, rubrics = gen_example()
  grader = NeutrinoV4WithBrowseGrader()
  rating = await grader(q, d, rubrics)
  print(rating)

await test_nv4_with_browse_grader()

2025-01-18 05:35:30 [warning  ] OTel metrics are not initialized, please initialize in your application by calling `otel.metrics.init()` bus=True
2025-01-18 05:35:31 [debug    ] Successfully subscribed to bus:client:bwen-quail-spart-grader-master-0-8jeRH9. bus=True
2025-01-18 05:35:32 [warning  ] %s: no more outstanding/pending requests; closing requestor. %s bus=True positional_args=('bus:client:bwen-quail-spart-grader-master-0-8jeRH9', '0 pending/0 outstanding/1 completed/0 errors')
2025-01-18 05:35:32 [debug    ] TopicManager(bus:client:bwen-quail-spart-grader-master-0-8jeRH9): 'CancelledError - Requestor(bus:client:bwen-quail-spart-grader-master-0-8jeRH9).close()' thrown; exiting. bus=True
2025-01-18 05:35:32 [debug    ] _Requestor.handle_responses(bus:client:bwen-quail-spart-grader-master-0-8jeRH9): 'CancelledError - Requestor(bus:client:bwen-quail-spart-grader-master-0-8jeRH9).close()' thrown; exiting. bus=True
2025-01-18 05:35:32 [debug    ] HeartbeatManager(bus:client:bwen-quai

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ content_type='system_error' name='ToolUsageError' text='`cursor` should not be set when opening a URL.'         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
Content Type: system_error
Channel: analysis
Weight: 1.0
End Turn: None



2025-01-18 05:35:32 [info     ] Model function call complete   component=function_calling_tool function_calling_tool={'tool_name': 'browser', 'call_id': 'fncall_b2338ec19ab34d38bf5461653feb656c', 'input_message': {'id': UUID('6b13b870-ad26-4f67-9944-2a5561aaa494'), 'author': {'role': <Role.ASSISTANT: 'assistant'>, 'name': None}, 'recipient': 'browser.open', 'channel': 'analysis', 'content': {'content_type': 'code', '__str__': '{"cursor": 0, "id": "https://www.theguardian.com/politics/2017/jun/15/controversial-lib-dem-donor-gave-200000-to-election-campaign"}'}}, 'response_message_ids': ['22b5bce8-e1f3-43da-818c-064e1ae9a619'], 'finished_successfully': True, 'function_calling_tool_version': '1.0.2', 'function_name': 'open', 'kwargs': {'cursor': 0, 'id': 'https://www.theguardian.com/politics/2017/jun/15/controversial-lib-dem-donor-gave-200000-to-election-campaign'}, '__fn_calling_tool_fn_type__': 'function_the_model_can_call'}
2025-01-18 05:35:32 [debug    ] Successfully subscribed to bus

assistant -> all
╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ## Relevance                                                                                                    │
│ * rating: Poor                                                                                                  │
│ * justification: The document is about a controversial Lib Dem donor giving money in recent campaigns (2015,    │
│ 2017) and mentions various donors but does not address who was the Liberal Democrats' largest donor in 2005. It │
│ is not relevant to the specific query.                                                                          │
│ * notes: The query is about 2005 and largest donor, while the article focuses on a £200,000 donation in a       │
│ different context.                                                                                              │
╰──────────────────────────────────────────────────────

# Data


In [ ]:
import os
import sys
from datetime import datetime, timedelta
from typing import Literal

import blobfile as bf
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import DataFrame, Window
from pyspark.sql.functions import (
    col,
    explode,
    from_json,
    struct,
    to_timestamp,
    unix_timestamp,
    when,
)
from pyspark.sql.types import ArrayType, BooleanType, FloatType, StringType, StructField, StructType

import chz
from maraschino_data_pipeline.lib.browse_utils import MinimalPageContents
from maraschino_data_pipeline.lib.file_utils import write_parquet_metadata
from maraschino_data_pipeline.lib.logging_utils import setup_logger
from maraschino_data_pipeline.lib.pipeline_context import PipelineContext
from maraschino_data_pipeline.lib.types import BingIndexType, LabradorIndexType, Source
from maraschino_data_pipeline.process_raw_data import generate_sonic_data, sample_by_create_date
from tinder.context import get_sq
from rosun import logging_utils, spark_utils
import logging
from tinder.context import maybe_az_to_wasb
import blobfile as bf

In [ ]:
input_patterns = [
    # "az://oaidsm1/oaimaraschino/datasets/citation/shuyangcheng/all_with_citation_full/sampled_data/all/sonic_prod_df_train/part*.parquet"
    "az://oaidsm1/oaibwen/data/citation/preprocess/query_generation_neutrino_10M/indexed_df/part-*.parquet"
]
ctx = spark_utils.new_ctx(input_patterns)

ctx.sc.setLogLevel("ERROR")

fns = []
for pattern in input_patterns:
    fns.extend(bf.glob(pattern))

fns = set(fns)
for p in input_patterns:
    fns.discard(p)

fns = sorted(fns)

print(f"Found {len(fns)} files")

raw_df = ctx.spark.read.parquet(*[maybe_az_to_wasb(f) for f in fns[:]])
raw_df = raw_df.cache()
print(f"Input table {spark_utils.formatted_schema(raw_df)}, {raw_df.count()} rows")

[tinder] Re-using existing SparkContext, new configuration may be ignored. Consider calling sc.stop() first
[tinder] Application UI at: http://go/spark-gui/spart-grader-master/bwen/quail


/root/.pyenv/versions/3.11.8/lib/python3.11/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Found 3000 files


Input table root
 |-- query: string (nullable = true)
 |-- user_metadata: string (nullable = true)
 |-- create_time: string (nullable = true)
 |-- rewritten_bing_queries: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- log_source: string (nullable = true)
 |-- search_result: struct (nullable = true)
 |    |-- url: string (nullable = true)
 |    |-- title: string (nullable = true)
 |    |-- text: string (nullable = true)
 |    |-- snippet_parts: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- snippet_scores: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- snippet_rendered: array (nullable = true)
 |    |    |-- element: boolean (containsNull = true)
 |    |-- position: integer (nullable = true)
 |    |-- pub_date: string (nullable = true)
 |    |-- score: double (nullable = true)
 |    |-- result_source: string (nullable = true)
 |    |-- result_type: string (nullable = true)
 |   

In [ ]:
raw_df.groupBy((F.col("deterministic_id") % 100).alias("bucket")).count().orderBy("bucket").show(100)

+------+-----+
|bucket|count|
+------+-----+
|  NULL|26247|
|     0|  252|
|     1|  262|
|     2|  259|
|     3|  289|
|     4|  262|
|     5|  294|
|     6|  271|
|     7|  254|
|     8|  265|
|     9|  283|
|    10|  253|
|    11|  255|
|    12|  273|
|    13|  254|
|    14|  278|
|    15|  290|
|    16|  269|
|    17|  293|
|    18|  273|
|    19|  269|
|    20|  292|
|    21|  246|
|    22|  289|
|    23|  274|
|    24|  275|
|    25|  248|
|    26|  270|
|    27|  266|
|    28|  259|
|    29|  278|
|    30|  261|
|    31|  280|
|    32|  236|
|    33|  231|
|    34|  261|
|    35|  274|
|    36|  277|
|    37|  247|
|    38|  262|
|    39|  237|
|    40|  276|
|    41|  247|
|    42|  265|
|    43|  262|
|    44|  238|
|    45|  282|
|    46|  308|
|    47|  248|
|    48|  267|
|    49|  279|
|    50|  257|
|    51|  255|
|    52|  241|
|    53|  260|
|    54|  244|
|    55|  252|
|    56|  267|
|    57|  270|
|    58|  250|
|    59|  250|
|    60|  284|
|    61|  302|
|    62|  

In [ ]:
filterd_df = raw_df.filter(
    ((F.col("deterministic_id") % 100) >= 7)
    & ((F.col("deterministic_id") % 100) < 10)
).count()
print(filterd_df)

802


In [ ]:
exploded_df = raw_df.withColumn("search_results", F.explode(F.col("search_results"))).where(F.col("search_results.status") == "ok")
exploded_df.cache()
exploded_df.select("search_results.search_result").show(3, truncate=True)



+--------------------+
|       search_result|
+--------------------+
|{"url":"https://w...|
|{"url":"https://w...|
|{"url":"https://w...|
+--------------------+
only showing top 3 rows



In [ ]:
from datetime import datetime
from search_service.api.user_metadata import UserMetadata
import json

def take_samples(df, n):
  res = df.select("query", "user_metadata", "create_time", "search_results.search_result").limit(n).collect()
  return res

def format_samples(samples, use_snippet=False):
  r = []
  for sample in samples:
    query = sample.query
    user_metadata = UserMetadata.parse_raw(sample.user_metadata)
    create_time = datetime.strptime(sample.create_time, "%Y-%m-%dT%H:%M:%S.%fZ")
    search_result = json.loads(sample.search_result)
    print(search_result.keys())

    q = Query(
        query=search_result["fetch_query"],
        create_timestamp=create_time,
        user_metadata=user_metadata,
        orig_query=sample.query,
    )



    if use_snippet:
      for snippet in search_result["snippet_parts"]:
        d = Document(
            url=search_result["url"],
            title=search_result["title"],
            content=snippet,
            pub_date=datetime.fromisoformat(search_result["publication_date"]),
        )
        r.append((q, d))
    else:
      d = Document(
          url=search_result["url"],
          title=search_result["title"],
          content=search_result["text"],
          pub_date=search_result["pub_date"],
      )
      r.append((q, d))
  return r


samples = take_samples(exploded_df, 10)
print(samples)

[Row(query='tell me everything about the LIFT ROI product. ', user_metadata='{"time_zone":"Europe/Copenhagen","user_locale":"en-GB","user_country":"DK","user_region":"Central Jutland","user_region_code":"82","cf_connecting_ip":"83.95.119.23","ip_city":"Risskov","latitude":null,"longitude":null,"locationAccuracy":null,"is_precise_location":false,"plan_type":"plus"}', create_time='2024-12-08T17:46:09.769Z', search_result='{"url":"https://www.kantar.com/campaigns/lift-roi","title":"LIFT ROI - Kantar","text":"In this article, How LIFT ROI works for Auto, you will learn how Kantar helps the automotive industry thrive and recover from the aftermath of supply chain issues, more people replacing air travel with cars, oil price daily/weekly fluctuation, etc. Auto manufacturers need granular insights on a daily rather than a yearly basis to compete in the rapidly changing, by the minute market.\\nLIFT ROI LIFT ROI Maximise your marketing effectiveness with AI-powered market mix modelling Book a 

# Rate!

In [ ]:
from pprint import pprint
parsed_samples = format_samples(samples)
print(
    pprint(parsed_samples[0][0].dict())
)
print(
    pprint(parsed_samples[0][1].dict())
)

dict_keys(['url', 'title', 'text', 'snippet_parts', 'snippet_scores', 'snippet_rendered', 'position', 'pub_date', 'score', 'result_source', 'result_type', 'is_cited', 'is_rendered', 'fetch_query'])
dict_keys(['url', 'title', 'text', 'snippet_parts', 'snippet_scores', 'snippet_rendered', 'position', 'pub_date', 'score', 'result_source', 'result_type', 'is_cited', 'is_rendered', 'fetch_query'])
dict_keys(['url', 'title', 'text', 'snippet_parts', 'snippet_scores', 'snippet_rendered', 'position', 'pub_date', 'score', 'result_source', 'result_type', 'is_cited', 'is_rendered', 'fetch_query'])
dict_keys(['url', 'title', 'text', 'snippet_parts', 'snippet_scores', 'snippet_rendered', 'position', 'pub_date', 'score', 'result_source', 'result_type', 'is_cited', 'is_rendered', 'fetch_query'])
dict_keys(['url', 'title', 'text', 'snippet_parts', 'snippet_scores', 'snippet_rendered', 'position', 'pub_date', 'score', 'result_source', 'result_type', 'is_cited', 'is_rendered', 'fetch_query'])
dict_keys(

In [ ]:
grader = ToolberryWithBrowseGrader()  # GeneralGrader(model="gpt-4o-mini")

r = await grader(query=parsed_samples[0][0], document=parsed_samples[0][1], rubrics="Rate the relevance of the the search result DOCUMENT to the user QUERY.")
print(r)

2025-01-17 22:50:18 [debug    ] Successfully subscribed to berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW. bus=True
2025-01-17 22:50:24 [warning  ] %s: no more outstanding/pending requests; closing requestor. %s bus=True positional_args=('berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW', '0 pending/0 outstanding/13 completed/0 errors')
2025-01-17 22:50:24 [debug    ] TopicManager(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW): 'CancelledError - Requestor(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW).close()' thrown; exiting. bus=True


2025-01-17 22:50:24 [debug    ] _Requestor.handle_responses(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW): 'CancelledError - Requestor(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW).close()' thrown; exiting. bus=True


2025-01-17 22:50:24 [debug    ] HeartbeatManager(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW:alive): 'CancelledError - HeartbeatManager(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW:alive).stop()' thrown; exiting. bus=True


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ content_type='system_error' name='ToolUsageError' text='No pages to access!'                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
Content Type: system_error
Channel: analysis
Weight: 1.0
End Turn: None



2025-01-17 22:50:24 [info     ] Model function call complete   component=function_calling_tool function_calling_tool={'tool_name': 'browser', 'call_id': 'fncall_631f09a8a2e04925a2407d7b0d259750', 'input_message': {'id': UUID('f8eb9e4e-b25b-4c6e-99e0-e143ac66b2a1'), 'author': {'role': <Role.ASSISTANT: 'assistant'>, 'name': None}, 'recipient': 'browser.open', 'channel': 'analysis', 'content': {'content_type': 'code', '__str__': '{"cursor": 0}'}}, 'response_message_ids': ['97fef326-97fa-46a9-ac0c-67fd0c865bf9'], 'finished_successfully': True, 'function_calling_tool_version': '1.0.2', 'function_name': 'open', 'kwargs': {'cursor': 0}, '__fn_calling_tool_fn_type__': 'function_the_model_can_call'}
2025-01-17 22:50:24 [debug    ] Successfully subscribed to berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW. bus=True
2025-01-17 22:50:25 [warning  ] %s: no more outstanding/pending requests; closing requestor. %s bus=True positional_args=('berry-shared:client:bwen-quail-spart-grader-mast

2025-01-17 22:50:25 [debug    ] TopicManager(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW): 'CancelledError - Requestor(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW).close()' thrown; exiting. bus=True


2025-01-17 22:50:25 [debug    ] _Requestor.handle_responses(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW): 'CancelledError - Requestor(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW).close()' thrown; exiting. bus=True
2025-01-17 22:50:25 [debug    ] HeartbeatManager(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW:alive): 'CancelledError - HeartbeatManager(berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW:alive).stop()' thrown; exiting. bus=True


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ content_type='tether_browsing_display' result="L0: # 【0†LIFT ROI - Kantar†www.kantar.com】\nL1: Discover       │
│ Kantar's AI-powered market mix modelling tool which brings together \nL2: data from sources inside and outside  │
│ of your business, including Kantar’s world-\nL3: class brand equity data, to give you near real-time access to  │
│ media and \nL4: marketing performance insights that enable you to make quick campaigns \nL5: optimisations and  │
│ accurately predict future scenarios.\nL6: \nL7: # 【1†LIFT ROI - kantar.com†www.kantar.com】\nL8: Having the    │
│ right tools and understanding of your unified marketing measurement, \nL9: puts you in control of your          │
│ marketing budget and allocation. We deploy a unique \nL10: econometric modelling approach to understand the     │
│ relative contribution of all \nL11: marketing mix elements, and their shor

2025-01-17 22:50:26 [info     ] Model function call complete   component=function_calling_tool function_calling_tool={'tool_name': 'browser', 'call_id': 'fncall_914a676e390249edbd82ece08a6f822f', 'input_message': {'id': UUID('52f7f21b-1750-40d1-b672-bde31b98d4f4'), 'author': {'role': <Role.ASSISTANT: 'assistant'>, 'name': None}, 'recipient': 'browser.search', 'channel': 'analysis', 'content': {'content_type': 'code', '__str__': '{"query": "\\"Kantar LIFT ROI\\""}'}}, 'response_message_ids': ['3a6ec1e6-5209-4a23-8cf6-e931c80dec82'], 'finished_successfully': True, 'function_calling_tool_version': '1.0.2', 'function_name': 'search', 'kwargs': {'query': '"Kantar LIFT ROI"'}, '__fn_calling_tool_fn_type__': 'function_the_model_can_call'}
2025-01-17 22:50:26 [debug    ] Successfully subscribed to berry-shared:client:bwen-quail-spart-grader-master-0-dvyFKW. bus=True
2025-01-17 22:50:27 [warning  ] %s: no more outstanding/pending requests; closing requestor. %s bus=True positional_args=('berry-

assistant -> all
╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ## Relevance                                                                                                    │
│ * rating: High                                                                                                  │
│ * justification: The document is a product page on Kantar's website titled "LIFT ROI - Kantar" and describes    │
│ Kantar’s LIFT ROI offering. The user query explicitly asks about the Kantar LIFT ROI product. The content       │
│ directly matches the query.                                                                                     │
│ * notes: None.                                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
Content Type: text
Channel: analysis
Weight: 1.0
messag

In [ ]:
from tool_use.browsing import fetch_with_caching, page_contents
from search_service.globals.aio import get_direct_session, get_proxied_session
from search_service import api
import aiohttp
from tool_use.browsing.labrador_browse_utils import open_url
import functools



async def fetch_page(url: str, timeout: float=1000) -> api.SonicPageContents:
  async with fetch_with_caching.create_proxied_http_client_session(
      cookie_jar=aiohttp.DummyCookieJar(),
  ) as session:
    fr = await fetch_with_caching.fetch(
        session,
        url,
        timeout=timeout,
        skip_robots_check=True,
        allowed_content_types_lengths=page_contents.ALLOWED_PAGE_CONTENT_TYPES_LENGTHS,
    )
    if not fr.success:
        raise Exception(f"Failed to fetch {url}: ({fr.error_type}) {fr.error_message}")

    if fr.content_type in ["text/html", "text/plain"]:
        if not fr.html:
            raise Exception(f"Failed to fetch {url}: ({fr.error_type}) {fr.error_message}")

        pc = await page_contents.process_html(
            html=fr.html,
            url=fr.url,
            title=fr.title,
            display_urls=True,
            display_dates=True,
            stats_tags=["tool_name:sonic"],
            content_type=fr.content_type,
        )
    elif fr.content_type in page_contents.ALLOWED_PAGE_CONTENT_TYPES_LENGTHS:
        if not fr.raw_content:
            raise Exception(f"Failed to fetch {url}: ({fr.error_type}) {fr.error_message}")

        pc = await page_contents.process_mime(
            raw_content=fr.raw_content,
            url=fr.url,
            title=fr.title,
            display_urls=True,
            display_dates=True,
            stats_tags=["tool_name:sonic"],
            content_type=fr.content_type,
        )
    else:
        raise Exception(f"Unsupposed content type from {url}: {fr.content_type}")

    if pc.failed:
        raise Exception(f"Failed to parse {url}: {pc.error_message}")
    return api.SonicPageContents(
        url=pc.url,
        title=pc.title,
        text=pc.text,
        content_type=pc.content_type,
        pub_date=date_str_to_timestamp(pc.pub_date),
        image_urls=pc.image_urls,
    )


async def fetch_labrador_page(url):
  async with aiohttp.ClientSession() as session:
    return await fetch_page(url, session)

pc = await fetch_page("https://www.wikiwand.com/en/articles/Marlow_Award")

# pc = await fetch_labrador_page("https://www.wikiwand.com/en/articles/Marlow_Award")
print(pc)

